# Top Brass (as compact as possible) -- Example 5.1 in Rardin (1998)

Top Brass Trophy Company makes large championship trophies for youth athletic leagues. At the moment, they are planning production for fall sports: football and soccer. Each football trophy has a wood base, an engraved plaque, a large brass football on top, and returns \\$12 in profit. Soccer trophies are similar (a brass ball, a wood base, and an engraved plaque) except that a brass soccer ball is on top, and the unit profit is only \\$9. Since the football has an asymmetric shape, its base requires 4 board feet of wood; the soccer base requires only 2 board feet. At the moment there are 1000 brass footballs in stock, 1500 soccer balls, 1750 plaques, and 4800 board feet of wood. What trophies should be produced from these supplies to maximize total profit assuming that all that are made can be sold?

## Problem Data

Like the modular version, we separate the data from the model structure. Now we also introduce the "NamedArray" object, which is a nice way to format arrays of data.

In [1]:
trophy_types = [:football, :soccer] # these are the possible trophy types

resources = [:wood, :plaques, :brass_football, :brass_soccer] # what goes into each trophy

profit = Dict( zip(trophy_types, [12,9] ) ) # profit produced by each trophy type

resource_avail = Dict( zip(resources, [4800, 1750, 1000, 1500] ) ); # amount of each type of resource available

# we use the NamedArrays package (you'll need to Pkg.add it first)
using NamedArrays

# create a matrix (Array) of the "recipe" for each trophy type. 
# each row is a trophy type, each column is a resource (should be ordered the same as the resources array).
# we read this as: "trophy type 1 (:football) requires 4 of resource 1 (:wood), 1 of resource 2 (:plaques),
# 1 of resource 3 (:brass_football), and 0 of resouce 4 (:brass_soccer)."
# row 2 is similar, but for trophy type :soccer.
trophy_resource_matrix = [4 1 1 0
                            2 1 0 1]

# create NamedArray that contains info on how much of each resource each trophy uses.
# syntax is (<"recipe" matrix>, (<row indices>,<column indices>),(<row name>,<column name>))
trophy_resource_NA = NamedArray(trophy_resource_matrix, (trophy_types, resources), ("type","resource"))

# check out the output to see how NamedArrays are structured:

2×4 Named Matrix{Int64}
type ╲ resource │           :wood  …    :brass_soccer
────────────────┼────────────────────────────────────
:football       │               4  …                0
:soccer         │               2  …                1

## Top Brass Model


In [2]:
# always specify which packages you're going to use
using JuMP, Clp

#create a new model object, specifying the solver
m = Model(Clp.Optimizer)

# trophy variable object is now a Dictionary indexed over trophy types (elements are variables)
@variable(m, trophy[trophy_types] >= 0)

# use an expression object to calculate the total profit
@expression(m, tot_profit, sum(profit[i] * trophy[i] for i in trophy_types) )

# our trophy/resource NamedArray allows us to create a Dictionary of constraints.
# indices are resources, and elements are constraint objects.
@constraint(m, constr[i in resources], sum(trophy_resource_NA[t, i] * trophy[t] for t in trophy_types) <= resource_avail[i] )

# our objective is to maximize the total profit
@objective(m, Max, tot_profit)

# solve the instance of the problem
optimize!(m)

# display solution information
println(value.(trophy))
println("Total profit will be \$", objective_value(m))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:football, :soccer]
And data, a 2-element Vector{Float64}:
  650.0
 1100.0
Total profit will be $17700.0
Coin0506I Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
Clp0006I 0  Obj -0 Dual inf 20.999998 (2)
Clp0006I 2  Obj 17700
Clp0000I Optimal - objective value 17700
Coin0511I After Postsolve, objective 17700, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 17700 - 2 iterations time 0.012, Presolve 0.00
